In [ ]:
import json
import os
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import pandas as pd
import folium
import ipinfo
import numpy as np
from datetime import datetime
from zipfile import ZipFile
import cbor2
from tqdm import tqdm


def get_cdf_data(values):
    sorted_values = np.sort(values)
    cdf = np.arange(len(sorted_values)) / len(sorted_values)
    return sorted_values, cdf

def is_float(s):
    try:
        float(s)
        return True
    except:
        return False

In [ ]:
lat_df = pd.read_csv("data/nesquic_latency.csv")
lat_df["fixed_latency"] = lat_df.apply(lambda r: 2*r["fixed_latency"], axis=1)
lat_df = lat_df.drop(columns=["Unnamed: 0"])

In [ ]:
lat_df

In [ ]:
lat_df["test"].unique()

In [ ]:
reno_df = lat_df.loc[lat_df["test"] == "bulk_upload_reno"]
bbr_df = lat_df.loc[lat_df["test"] == "bulk_upload_bbr"]
cubic_df = lat_df.loc[lat_df["test"] == "bulk_upload_cubic"]
fast_df = lat_df.loc[lat_df["test"] == "bulk_upload_fast"]
limit3_df = lat_df.loc[lat_df["test"] == "limited_upload_0.3"]
limit5_df = lat_df.loc[lat_df["test"] == "limited_upload_0.5"]
limit8_df = lat_df.loc[lat_df["test"] == "limited_upload_0.8"]
limit9_df = lat_df.loc[lat_df["test"] == "limited_upload_0.9"]
limit10_df = lat_df.loc[lat_df["test"] == "limited_upload_1.0"]
limit11_df = lat_df.loc[lat_df["test"] == "limited_upload_1.1"]

In [ ]:
tmp_fast = fast_df[["min", "mean", "median", "0.8", "0.9"]]
z_scores = np.abs((tmp_fast - tmp_fast.mean()) / tmp_fast.std())
fast_df = fast_df[(z_scores < 3).all(axis=1)]

In [ ]:
def filter_z_score(list, treshold=3):
    mean = np.mean(list)
    std = np.std(list)
    return [e for e in list if np.abs((e - mean) / std) < treshold]

## CCA

In [ ]:
def plot_cca(wtp):
    fig, axs = plt.subplots(2, 2)

    axs[0, 0].scatter(bbr_df["fixed_bw"], bbr_df["fixed_latency"],
                      c=bbr_df[wtp])
    axs[0, 0].set_title("BBR")
    axs[0, 0].set_xticks([])

    axs[0, 1].scatter(cubic_df["fixed_bw"], cubic_df["fixed_latency"],
                      c=cubic_df[wtp])
    axs[0, 1].set_title("Cubic")
    axs[0, 1].set_xticks([])
    axs[0, 1].set_yticks([])

    axs[1, 0].scatter(reno_df["fixed_bw"], reno_df["fixed_latency"],
                      c=reno_df[wtp])
    axs[1, 0].set_title("New Reno")

    sca = axs[1, 1].scatter(fast_df["fixed_bw"],
                            fast_df["fixed_latency"], c=fast_df[wtp])
    axs[1, 1].set_title("Fast")
    axs[1, 1].set_yticks([])

    cax = fig.add_axes([0.85, 0.1, 0.02, 0.7])

    cbar = plt.colorbar(sca, cax=cax)
    cbar.set_label(f"Measured {wtp} RTT (ms)")

    fig.supylabel("Fixed RTT (ms)")
    fig.supxlabel("Fixed bandwidth (Mbps)")
    plt.subplots_adjust(wspace=0.04, hspace=0.2,
                        top=0.8, bottom=0.12, right=0.80)
    # plt.savefig("output/increase_loss.pdf")


In [ ]:
plot_cca("min")
plt.savefig("output/NesQUIClab/bulk_lat_min.pdf")


In [ ]:
plot_cca("median")

In [ ]:
plot_cca("mean")
plt.savefig("output/NesQUIClab/bulk_lat_mean.pdf")


In [ ]:
plot_cca("0.9")
plt.savefig("output/NesQUIClab/bulk_lat_9.pdf")


In [ ]:

fig, axs = plt.subplots(2, 2, sharey=True)

for metric in ["mean", "median", "0.8", "0.9"]:
    label = f"{int(float(metric)*100)} percentile" if is_float(metric) else metric

    d, c = get_cdf_data(bbr_df[metric])
    axs[0, 0].step(d, c, label=label)
    d, c = get_cdf_data(cubic_df[metric])
    axs[0, 1].step(d, c, label=label)
    d, c = get_cdf_data(reno_df[metric])
    axs[1, 0].step(d, c, label=label)
    d, c = get_cdf_data(fast_df[metric])
    axs[1, 1].step(d, c, label=label)

axs[0, 0].set_title("BBR")

axs[0, 1].set_title("Cubic")
# axs[0, 1].set_yticks([])

axs[1, 0].set_title("New Reno")

axs[1, 1].set_title("Fast")
# axs[1, 1].set_yticks([])

axs[0, 0].grid()
axs[0, 1].grid()
axs[1, 0].grid()
axs[1, 1].grid()
axs[1, 1].set_xlim(xmin=0, xmax=400)
leg = plt.legend(title="Metrics", bbox_to_anchor=(1, 1.6))

fig.supxlabel("RTT (ms)")
fig.supylabel("Cumulative Probability")
plt.subplots_adjust(wspace=0.04, hspace=0.5, top=0.8, bottom=0.12, right=0.75)
plt.savefig("output/NesQUIClab/bulk_lat_cdf.pdf", bbox_extra_artists=(leg,))

## On metrics

In [ ]:
def limit_sca(wtp):
    plt.scatter(8*limit11_df["fixed_bw"]*limit11_df["fixed_latency"]/1000,
                limit11_df[wtp], label="1.1")
    plt.scatter(8*limit10_df["fixed_bw"]*limit10_df["fixed_latency"]/1000,
                limit10_df["median"], label="1.0")
    plt.scatter(8*limit9_df["fixed_bw"]*limit9_df["fixed_latency"]/1000,
                limit9_df[wtp], label="0.9")
    plt.scatter(8*limit8_df["fixed_bw"]*limit8_df["fixed_latency"]/1000,
                limit8_df[wtp], label="0.8")
    plt.scatter(8*limit5_df["fixed_bw"]*limit5_df["fixed_latency"]/1000,
                limit5_df[wtp], label="0.5")
    plt.scatter(8*limit3_df["fixed_bw"]*limit3_df["fixed_latency"]/1000,
                limit3_df[wtp], label="0.3")
    plt.legend(bbox_to_anchor=(1, 1.04))
    plt.xlabel("BDP (kB)")
    plt.ylabel(f"{wtp} RTT (ms)")

In [ ]:
limit_sca("mean")

In [ ]:
limit_sca("median")

In [ ]:
limit_sca("0.9")

In [ ]:
limit_sca("min")

## On tests

In [ ]:
limited_tests = [t for t in lat_df.test.unique() if t.startswith("limited")]

flemme = {
    0: (0, 0),
    1: (0, 1),
    2: (1, 0),
    3: (1, 1),
    4: (2, 0),
    5: (2, 1),
}

fig, axs = plt.subplots(3, 2, 
                        # sharex=True, 
                        # sharey=True
                        )
# for i in range(3):
#     axs[i, 0].get_shared_y_axes().join(axs[i, 0], axs[i, 1])

for i, test_name in enumerate(limited_tests):
    c_df = lat_df.loc[lat_df["test"] == test_name]
    row, col = flemme[i]
    for metric in ["mean", "median", "0.8", "0.9", "0.99"]:
        d, c = get_cdf_data(filter_z_score(c_df[metric]))
        label = f"{int(float(metric)*100)} percentile" if is_float(metric) else metric
        axs[row, col].step(d, c, label=label)
    title = f"{int(100*float(test_name.split('_')[-1]))}% of estimated bandwidth"
    axs[row, col].set_title(title)
    axs[row, col].grid()
    if col != 0: axs[row, col].set_yticks([])
    # if row != 2: axs[row, col].set_xticks([])
axs[0, 0].set_xlim(xmin=-.1, xmax=500)
axs[1, 0].set_xlim(xmin=-.1, xmax=500)
fig.supylabel("Cumulative Probability")
xlab = fig.supxlabel("RTT (ms)")
lgd = plt.legend(title="Metrics", bbox_to_anchor=(1.7, 3))
plt.subplots_adjust(wspace=0.04, hspace=0.6, top=0.8, bottom=0.12, left=.1)
plt.savefig("output/NesQUIClab/limited_lat_metrics_cdf.pdf", bbox_extra_artists=(lgd,xlab), bbox_inches='tight')

In [ ]:
limited_tests = [t for t in lat_df.test.unique() if t.startswith("limited")]

flemme = {
    0: (0, 0),
    1: (0, 1),
    2: (1, 0),
    3: (1, 1),
    4: (2, 0),
    5: (2, 1),
}

fig, axs = plt.subplots(3, 2, 
                        sharex=True, 
                        # sharey=True
                        )
for i in range(3):
    axs[i, 0].get_shared_y_axes().join(axs[i, 0], axs[i, 1])

for i, test_name in enumerate(limited_tests):
    c_df = lat_df.loc[lat_df["test"] == test_name]
    row, col = flemme[i]
    for metric in ["mean", "median", "0.8", "0.9", "0.99"]:
        axs[row, col].scatter(8*c_df["fixed_bw"]*c_df["fixed_latency"]/1000,
                              c_df[metric], label=metric)
    axs[row, col].set_title(test_name)
    if col != 0: axs[row, col].set_yticks([])
    # if row != 2: axs[row, col].set_xticks([])
axs[0, 0].set_ylim(ymin=-.1, ymax=500)
axs[1, 0].set_ylim(ymin=-.1, ymax=500)
fig.supylabel("Measured RTT (ms)")
xlab = fig.supxlabel("BDP (kB)")
lgd = plt.legend(bbox_to_anchor=(1.5, 3))
plt.subplots_adjust(wspace=0.04, hspace=0.4, top=0.8, bottom=0.12, left=.1)
plt.savefig("output/NesQUIClab/limited_lat_metrics.pdf", bbox_extra_artists=(lgd,xlab), bbox_inches='tight')

In [ ]:
def plot_rtt(df):
    for col in ["min", "mean", "median", "0.8", "0.9", "0.99"]:
        plt.scatter(8*df["fixed_bw"]*df["fixed_latency"]/1000,
                    df[col], label=col)
    plt.legend(bbox_to_anchor=(1, 1.04))
    plt.xlabel("BDP (kB)")
    plt.ylabel("RTT (ms)")

In [ ]:
plot_rtt(limit11_df)